# Baseline Models - Implementation

Simple reference baselines for benchmarking:
- GlobalMean: League average
- TeamMean: Per-team averages
- HomeAway: Location-aware stats
- MovingAverage: Recent N games
- WeightedHistory: Exponential decay
- Poisson: Statistical model

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('darkgrid')
plt.rcParams['figure.figsize'] = (12, 6)

## Column Aliases

Handles different column naming conventions in datasets.

In [ ]:
COLUMN_ALIASES = {
    'home_team': ['home_team', 'home', 'team_home', 'h_team'],
    'away_team': ['away_team', 'away', 'team_away', 'a_team', 'visitor', 'visiting_team'],
    'home_goals': ['home_goals', 'home_score', 'h_goals', 'goals_home', 'home_pts'],
    'away_goals': ['away_goals', 'away_score', 'a_goals', 'goals_away', 'away_pts', 'visitor_goals'],
    'game_date': ['game_date', 'date', 'Date', 'game_datetime', 'datetime', 'game_time'],
}


def get_value(game, field, default=None):
    """Get a value from a game record, checking multiple possible column names."""
    aliases = COLUMN_ALIASES.get(field, [field])
    for alias in aliases:
        if alias in game:
            val = game[alias]
            if pd.isna(val):
                return default
            return val
    return default


def get_column(df, field):
    """Find the correct column name in a DataFrame."""
    aliases = COLUMN_ALIASES.get(field, [field])
    for alias in aliases:
        if alias in df.columns:
            return alias
    return None

## Base Model Class

In [ ]:
class BaselineModel:
    """
    Abstract base class for all baseline models.
    
    All baseline models must implement:
        - fit(games_df): Train on historical data
        - predict_goals(game): Predict (home_goals, away_goals)
    """
    
    def __init__(self, params=None):
        self.params = params or {}
        self.is_fitted = False
    
    def fit(self, games_df):
        """Train the model on historical games."""
        raise NotImplementedError("Subclass must implement fit()")
    
    def predict_goals(self, game):
        """Predict goals for a game. Returns (home_goals, away_goals)."""
        raise NotImplementedError("Subclass must implement predict_goals()")
    
    def evaluate(self, games_df):
        """Evaluate model on test set."""
        if not self.is_fitted:
            raise RuntimeError("Model must be fitted before evaluation")
        
        home_preds, away_preds = [], []
        home_actuals, away_actuals = [], []
        
        for _, game in games_df.iterrows():
            home_pred, away_pred = self.predict_goals(game)
            home_preds.append(home_pred)
            away_preds.append(away_pred)
            home_actuals.append(get_value(game, 'home_goals', 0))
            away_actuals.append(get_value(game, 'away_goals', 0))
        
        # Home goals metrics
        rmse = mean_squared_error(home_actuals, home_preds, squared=False)
        mae = mean_absolute_error(home_actuals, home_preds)
        r2 = r2_score(home_actuals, home_preds) if len(set(home_actuals)) > 1 else 0.0
        
        # Combined metrics
        all_preds = home_preds + away_preds
        all_actuals = home_actuals + away_actuals
        combined_rmse = mean_squared_error(all_actuals, all_preds, squared=False)
        
        return {
            'rmse': rmse,
            'mae': mae,
            'r2': r2,
            'combined_rmse': combined_rmse
        }
    
    def get_summary(self):
        """Return model summary dict."""
        return {'model': self.__class__.__name__, 'params': self.params}

## GlobalMeanBaseline

Predicts league-wide average for all games. The simplest possible baseline.

In [ ]:
class GlobalMeanBaseline(BaselineModel):
    """
    Predict the league-wide average goals for all games.
    
    This is the absolute minimum baseline. Any model that can't
    beat GlobalMean is essentially useless.
    """
    
    def fit(self, games_df):
        home_col = get_column(games_df, 'home_goals')
        away_col = get_column(games_df, 'away_goals')
        
        self.global_mean_home = games_df[home_col].mean()
        self.global_mean_away = games_df[away_col].mean()
        self.n_games = len(games_df)
        self.is_fitted = True
        return self
    
    def predict_goals(self, game):
        return self.global_mean_home, self.global_mean_away
    
    def get_summary(self):
        return {
            'model': 'GlobalMeanBaseline',
            'global_mean_home': round(self.global_mean_home, 3),
            'global_mean_away': round(self.global_mean_away, 3),
            'n_games': self.n_games
        }

## TeamMeanBaseline

Predicts based on team-specific offensive and defensive averages.

In [ ]:
class TeamMeanBaseline(BaselineModel):
    """
    Predict based on team-specific offensive/defensive averages.
    
    home_pred = (home_team_offense + away_team_defense) / 2
    
    This is the standard baseline for sports prediction.
    """
    
    def fit(self, games_df):
        home_team_col = get_column(games_df, 'home_team')
        away_team_col = get_column(games_df, 'away_team')
        home_goals_col = get_column(games_df, 'home_goals')
        away_goals_col = get_column(games_df, 'away_goals')
        
        goals_for = {}
        goals_against = {}
        games_played = {}
        
        for _, game in games_df.iterrows():
            home_team = game[home_team_col]
            away_team = game[away_team_col]
            home_goals = game[home_goals_col]
            away_goals = game[away_goals_col]
            
            # Home team stats
            goals_for[home_team] = goals_for.get(home_team, 0) + home_goals
            goals_against[home_team] = goals_against.get(home_team, 0) + away_goals
            games_played[home_team] = games_played.get(home_team, 0) + 1
            
            # Away team stats
            goals_for[away_team] = goals_for.get(away_team, 0) + away_goals
            goals_against[away_team] = goals_against.get(away_team, 0) + home_goals
            games_played[away_team] = games_played.get(away_team, 0) + 1
        
        self.team_offense = {t: goals_for[t] / games_played[t] for t in games_played}
        self.team_defense = {t: goals_against[t] / games_played[t] for t in games_played}
        self.global_mean = games_df[home_goals_col].mean()
        self.n_teams = len(games_played)
        self.is_fitted = True
        return self
    
    def predict_goals(self, game):
        home_team = get_value(game, 'home_team')
        away_team = get_value(game, 'away_team')
        
        home_off = self.team_offense.get(home_team, self.global_mean)
        home_def = self.team_defense.get(home_team, self.global_mean)
        away_off = self.team_offense.get(away_team, self.global_mean)
        away_def = self.team_defense.get(away_team, self.global_mean)
        
        home_pred = (home_off + away_def) / 2
        away_pred = (away_off + home_def) / 2
        
        return home_pred, away_pred
    
    def get_summary(self):
        return {
            'model': 'TeamMeanBaseline',
            'n_teams': self.n_teams,
            'global_mean': round(self.global_mean, 3)
        }

## HomeAwayBaseline

Tracks separate statistics for home and away games.

In [ ]:
class HomeAwayBaseline(BaselineModel):
    """
    Account for home/away goal differentials.
    
    Teams often perform differently at home vs away.
    This baseline captures that pattern.
    """
    
    def fit(self, games_df):
        home_team_col = get_column(games_df, 'home_team')
        away_team_col = get_column(games_df, 'away_team')
        home_goals_col = get_column(games_df, 'home_goals')
        away_goals_col = get_column(games_df, 'away_goals')
        
        home_goals_for, home_goals_against, home_games = {}, {}, {}
        away_goals_for, away_goals_against, away_games = {}, {}, {}
        
        for _, game in games_df.iterrows():
            ht, at = game[home_team_col], game[away_team_col]
            hg, ag = game[home_goals_col], game[away_goals_col]
            
            home_goals_for[ht] = home_goals_for.get(ht, 0) + hg
            home_goals_against[ht] = home_goals_against.get(ht, 0) + ag
            home_games[ht] = home_games.get(ht, 0) + 1
            
            away_goals_for[at] = away_goals_for.get(at, 0) + ag
            away_goals_against[at] = away_goals_against.get(at, 0) + hg
            away_games[at] = away_games.get(at, 0) + 1
        
        self.home_offense = {t: home_goals_for[t]/home_games[t] for t in home_games}
        self.home_defense = {t: home_goals_against[t]/home_games[t] for t in home_games}
        self.away_offense = {t: away_goals_for[t]/away_games[t] for t in away_games}
        self.away_defense = {t: away_goals_against[t]/away_games[t] for t in away_games}
        
        self.global_home_mean = games_df[home_goals_col].mean()
        self.global_away_mean = games_df[away_goals_col].mean()
        self.is_fitted = True
        return self
    
    def predict_goals(self, game):
        ht = get_value(game, 'home_team')
        at = get_value(game, 'away_team')
        
        home_off = self.home_offense.get(ht, self.global_home_mean)
        away_def = self.away_defense.get(at, self.global_home_mean)
        away_off = self.away_offense.get(at, self.global_away_mean)
        home_def = self.home_defense.get(ht, self.global_away_mean)
        
        return (home_off + away_def) / 2, (away_off + home_def) / 2
    
    def get_summary(self):
        return {
            'model': 'HomeAwayBaseline',
            'home_advantage': round(self.global_home_mean - self.global_away_mean, 3)
        }

## MovingAverageBaseline

Uses only the last N games for predictions (captures form).

In [ ]:
class MovingAverageBaseline(BaselineModel):
    """
    Use only the last N games for predictions.
    
    Captures "team form" - recent performance matters more.
    
    Hyperparameters:
        window: Number of recent games (default: 5)
    """
    
    def __init__(self, params=None):
        super().__init__(params)
        self.window = self.params.get('window', 5)
    
    def fit(self, games_df):
        home_team_col = get_column(games_df, 'home_team')
        away_team_col = get_column(games_df, 'away_team')
        home_goals_col = get_column(games_df, 'home_goals')
        away_goals_col = get_column(games_df, 'away_goals')
        
        self.team_history = {}
        
        for _, game in games_df.iterrows():
            ht, at = game[home_team_col], game[away_team_col]
            hg, ag = game[home_goals_col], game[away_goals_col]
            
            if ht not in self.team_history:
                self.team_history[ht] = []
            if at not in self.team_history:
                self.team_history[at] = []
            
            self.team_history[ht].append((hg, ag))  # (goals_for, goals_against)
            self.team_history[at].append((ag, hg))
        
        self.global_mean = games_df[home_goals_col].mean()
        self.is_fitted = True
        return self
    
    def _get_recent_avg(self, team):
        if team not in self.team_history or len(self.team_history[team]) == 0:
            return self.global_mean, self.global_mean
        recent = self.team_history[team][-self.window:]
        goals_for = np.mean([g[0] for g in recent])
        goals_against = np.mean([g[1] for g in recent])
        return goals_for, goals_against
    
    def predict_goals(self, game):
        ht = get_value(game, 'home_team')
        at = get_value(game, 'away_team')
        
        home_off, home_def = self._get_recent_avg(ht)
        away_off, away_def = self._get_recent_avg(at)
        
        return (home_off + away_def) / 2, (away_off + home_def) / 2
    
    def get_summary(self):
        return {'model': f'MovingAverage(window={self.window})', 'window': self.window}

## WeightedHistoryBaseline

Recent games count more than older games (exponential decay).

In [ ]:
class WeightedHistoryBaseline(BaselineModel):
    """
    Recent games count more than older games.
    
    Uses exponential decay: weight = decay^(games_ago)
    
    Hyperparameters:
        decay: Decay factor, 0.7-0.99 (default: 0.9)
    """
    
    def __init__(self, params=None):
        super().__init__(params)
        self.decay = self.params.get('decay', 0.9)
    
    def fit(self, games_df):
        home_team_col = get_column(games_df, 'home_team')
        away_team_col = get_column(games_df, 'away_team')
        home_goals_col = get_column(games_df, 'home_goals')
        away_goals_col = get_column(games_df, 'away_goals')
        
        self.team_history = {}
        
        for _, game in games_df.iterrows():
            ht, at = game[home_team_col], game[away_team_col]
            hg, ag = game[home_goals_col], game[away_goals_col]
            
            if ht not in self.team_history:
                self.team_history[ht] = []
            if at not in self.team_history:
                self.team_history[at] = []
            
            self.team_history[ht].append((hg, ag))
            self.team_history[at].append((ag, hg))
        
        self.global_mean = games_df[home_goals_col].mean()
        self.is_fitted = True
        return self
    
    def _get_weighted_avg(self, team):
        if team not in self.team_history or len(self.team_history[team]) == 0:
            return self.global_mean, self.global_mean
        
        history = self.team_history[team]
        n = len(history)
        weighted_for, weighted_against, total_weight = 0, 0, 0
        
        for i, (gf, ga) in enumerate(history):
            weight = self.decay ** (n - 1 - i)  # Most recent has highest weight
            weighted_for += gf * weight
            weighted_against += ga * weight
            total_weight += weight
        
        return weighted_for / total_weight, weighted_against / total_weight
    
    def predict_goals(self, game):
        ht = get_value(game, 'home_team')
        at = get_value(game, 'away_team')
        
        home_off, home_def = self._get_weighted_avg(ht)
        away_off, away_def = self._get_weighted_avg(at)
        
        return (home_off + away_def) / 2, (away_off + home_def) / 2
    
    def get_summary(self):
        return {'model': f'WeightedHistory(decay={self.decay})', 'decay': self.decay}

## PoissonBaseline

Statistical Poisson regression model - the academic standard.

In [ ]:
class PoissonBaseline(BaselineModel):
    """
    Statistical Poisson regression model.
    
    Goals follow a Poisson distribution. This baseline estimates:
    - Attack strength per team
    - Defense strength per team
    - Home field advantage factor
    
    Prediction: league_avg * attack * opponent_defense * home_factor
    """
    
    def fit(self, games_df):
        home_team_col = get_column(games_df, 'home_team')
        away_team_col = get_column(games_df, 'away_team')
        home_goals_col = get_column(games_df, 'home_goals')
        away_goals_col = get_column(games_df, 'away_goals')
        
        self.league_avg = games_df[home_goals_col].mean()
        self.home_factor = games_df[home_goals_col].mean() / max(games_df[away_goals_col].mean(), 0.01)
        
        goals_for, goals_against, games_played = {}, {}, {}
        
        for _, game in games_df.iterrows():
            ht, at = game[home_team_col], game[away_team_col]
            hg, ag = game[home_goals_col], game[away_goals_col]
            
            goals_for[ht] = goals_for.get(ht, 0) + hg
            goals_against[ht] = goals_against.get(ht, 0) + ag
            games_played[ht] = games_played.get(ht, 0) + 1
            
            goals_for[at] = goals_for.get(at, 0) + ag
            goals_against[at] = goals_against.get(at, 0) + hg
            games_played[at] = games_played.get(at, 0) + 1
        
        # Attack strength = team avg / league avg
        self.attack_strength = {
            t: (goals_for[t] / games_played[t]) / self.league_avg 
            for t in games_played
        }
        
        # Defense strength = goals allowed avg / league avg
        self.defense_strength = {
            t: (goals_against[t] / games_played[t]) / self.league_avg 
            for t in games_played
        }
        
        self.is_fitted = True
        return self
    
    def predict_goals(self, game):
        ht = get_value(game, 'home_team')
        at = get_value(game, 'away_team')
        
        home_att = self.attack_strength.get(ht, 1.0)
        home_def = self.defense_strength.get(ht, 1.0)
        away_att = self.attack_strength.get(at, 1.0)
        away_def = self.defense_strength.get(at, 1.0)
        
        home_goals = self.league_avg * home_att * away_def * self.home_factor
        away_goals = self.league_avg * away_att * home_def / self.home_factor
        
        return home_goals, away_goals
    
    def get_summary(self):
        return {
            'model': 'PoissonBaseline',
            'league_avg': round(self.league_avg, 3),
            'home_factor': round(self.home_factor, 3)
        }

## Compare All Baselines

In [ ]:
def compare_baselines(games_df, test_df=None, models=None):
    """
    Train and compare multiple baseline models.
    
    Args:
        games_df: Training data
        test_df: Test data (optional, uses train if None)
        models: List of model instances (optional, uses all if None)
    
    Returns:
        DataFrame with comparison results
    """
    if test_df is None:
        test_df = games_df
    
    if models is None:
        models = [
            GlobalMeanBaseline(),
            TeamMeanBaseline(),
            HomeAwayBaseline(),
            MovingAverageBaseline({'window': 3}),
            MovingAverageBaseline({'window': 5}),
            MovingAverageBaseline({'window': 10}),
            WeightedHistoryBaseline({'decay': 0.85}),
            WeightedHistoryBaseline({'decay': 0.90}),
            WeightedHistoryBaseline({'decay': 0.95}),
            PoissonBaseline()
        ]
    
    results = []
    for model in models:
        model.fit(games_df)
        metrics = model.evaluate(test_df)
        summary = model.get_summary()
        results.append({
            'model': summary.get('model', model.__class__.__name__),
            'rmse': round(metrics['rmse'], 4),
            'mae': round(metrics['mae'], 4),
            'r2': round(metrics['r2'], 4),
            'combined_rmse': round(metrics['combined_rmse'], 4)
        })
    
    return pd.DataFrame(results).sort_values('rmse')

## Example Usage

In [ ]:
# Load data (replace with your actual data path)
# df = pd.read_csv('data/hockey_data.csv')
# df = df.sort_values('game_date')  # CRITICAL: chronological order

# Split 80/20 chronologically
# split_idx = int(len(df) * 0.8)
# train_df = df.iloc[:split_idx]
# test_df = df.iloc[split_idx:]

# Compare all baselines
# results = compare_baselines(train_df, test_df)
# print(results)

# Or train a single model
# model = TeamMeanBaseline()
# model.fit(train_df)
# metrics = model.evaluate(test_df)
# print(f"RMSE: {metrics['rmse']:.4f}")

## Visualize Comparison

In [ ]:
# Plot RMSE comparison
# plt.figure(figsize=(12, 6))
# colors = ['green' if x == results['rmse'].min() else 'steelblue' for x in results['rmse']]
# plt.barh(results['model'], results['rmse'], color=colors)
# plt.xlabel('RMSE (lower is better)')
# plt.title('Baseline Model Comparison')
# plt.gca().invert_yaxis()
# plt.tight_layout()
# plt.show()